In [2]:
%matplotlib inline
import matplotlib
import seaborn as sns
sns.set()
matplotlib.rcParams['figure.dpi'] = 144

In [3]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [4]:
from static_grader import grader

# ML Miniproject
## Introduction

The objective of this miniproject is to exercise your ability to create effective machine learning models for making predictions. We will be working with nursing home inspection data from the United States, predicting which providers may be fined and for how much.

## Scoring

In this miniproject you will often submit your model's `predict` or `predict_proba` method to the grader. The grader will assess the performance of your model using a scoring metric, comparing it against the score of a reference model. We will use the [average precision score](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.average_precision_score.html). If your model performs better than the reference solution, then you can score higher than 1.0.

**Note:** If you use an estimator that relies on random draws (like a `RandomForestClassifier`) you should set the `random_state=` to an integer so that your results are reproducible. 

## Downloading the data

We can download the data set from Amazon S3:

In [5]:
%%bash
mkdir data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-train.csv -nc -P ./ml-data
wget http://dataincubator-wqu.s3.amazonaws.com/mldata/providers-metadata.csv -nc -P ./ml-data

mkdir: cannot create directory ‘data’: File exists
File ‘./ml-data/providers-train.csv’ already there; not retrieving.

File ‘./ml-data/providers-metadata.csv’ already there; not retrieving.



We'll load the data into a Pandas DataFrame. Several columns will become target labels in future questions. Let's pop those columns out from the data, and drop related columns that are neither targets nor reasonable features (i.e. we don't wouldn't know how many times a facility denied payment before knowing whether it was fined).

The data has many columns. We have also provided a data dictionary.

In [6]:
import numpy as np
import pandas as pd

In [7]:
metadata = pd.read_csv('./ml-data/providers-metadata.csv')
metadata.head()

,Variable,Label,Description,Format
0,PROVNUM,Federal Provider Number,Federal Provider Number,6 alphanumeric characters
1,PROVNAME,Provider Name,Provider Name,text
2,ADDRESS,Provider Address,Provider Address,text
3,CITY,Provider City,Provider City,text
4,STATE,Provider State,Provider State,2-character postal abbreviation


In [8]:
data = pd.read_csv('./ml-data/providers-train.csv', encoding='latin1')

fine_counts = data.pop('FINE_CNT')
fine_totals = data.pop('FINE_TOT')
cycle_2_score = data.pop('CYCLE_2_TOTAL_SCORE')

In [8]:
data.head()

,PROVNUM,PROVNAME,ADDRESS,CITY,STATE,ZIP,PHONE,COUNTY_SSA,COUNTY_NAME,BEDCERT,...,CERTIFICATION,CYCLE_1_DEFS,CYCLE_1_NFROMDEFS,CYCLE_1_NFROMCOMP,CYCLE_1_DEFS_SCORE,CYCLE_1_NUMREVIS,CYCLE_1_REVISIT_SCORE,CYCLE_1_TOTAL_SCORE,CYCLE_1_SURVEY_DATE,CYCLE_2_SURVEY_DATE
0,015010,COOSA VALLEY NURSING FACILITY,315 WEST HICKORY STREET,SYLACAUGA,AL,35150,2562495604,600,Talladega,85,...,Medicare and Medicaid,7,7,0,36,1,0,36,2017-04-06,2016-05-26
1,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,350,Jackson,50,...,Medicare and Medicaid,5,5,0,44,1,0,44,2017-03-16,2016-02-04
2,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,360,Jefferson,92,...,Medicare and Medicaid,6,6,0,40,1,0,40,2016-10-20,2015-12-30
3,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY P O BOX 97,MC CALLA,AL,35111,2054776161,360,Jefferson,103,...,Medicare and Medicaid,2,2,0,16,1,0,16,2017-03-09,2016-02-11
4,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,410,Limestone,149,...,Medicare and Medicaid,2,2,0,20,1,0,20,2017-06-01,2016-05-12


## Question 1: state_model

A federal agency, Centers for Medicare and Medicaid Services (CMS), imposes regulations on nursing homes. However, nursing homes are inspected by state agencies for compliance with regulations, and fines for violations can vary widely between states.

Let's develop a very simple initial model to predict the amount of fines a nursing home might expect to pay based on its location. Fill in the class definition of the custom estimator, `StateMeanEstimator`, below.

In [9]:
state_avgs = fine_totals.groupby(data['STATE']).mean().to_dict()
# mean fines for each state
# interesting thing - we here group by not what is in fine_totals but 
# by what is in the STATE column in data! this is possible because they share the same indices 
# (because fine_totals is a series from data)
# to_dict converts a series into a dict

In [9]:
from sklearn.base import BaseEstimator, RegressorMixin, TransformerMixin

class GroupMeanEstimator(BaseEstimator, RegressorMixin):
    def __init__(self, grouper):
        self.grouper = grouper
        self.group_averages = {}
        self.global_average = 0

    def fit(self, X, y): 
        self.group_averages = y.groupby(X[self.grouper]).mean().to_dict()
        # we change our earlier state_avgs into a more general formula
        # data is X here, fine_totals is y, STATE series is self.grouper
        self.global_average = y.mean()
        # Use self.group_averages to store the average penalty by group
        return self

    def predict(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        # Return a list of predicted penalties based on group of samples in X
        return [self.group_averages.get(group, self.global_average) for group in X[self.grouper]]

In [11]:
fine_totals.get('RU', 4) #metoda get() sprawdza wartość w słowniku, a jeśli klucz nie istnieje - zwraca default

4

After filling in class definition, we can create an instance of the estimator and fit it to the data.

In [10]:
from sklearn.pipeline import Pipeline

state_model = Pipeline([
    ('sme', GroupMeanEstimator(grouper='STATE'))
    ])
state_model.fit(data, fine_totals)

Pipeline(memory=None, steps=[('sme', GroupMeanEstimator(grouper='STATE'))],
         verbose=False)

Next we should test that our predict method works.

In [11]:
state_model.predict(data.sample(5))

[16565.303571428572,
 16722.25925925926,
 8054.977611940299,
 3219.326530612245,
 1512.7222222222222]

However, what if we have data from a nursing home in a state (or territory) of the US which is not in the training data?

In [13]:
state_model.predict(pd.DataFrame([{'STATE': 'AS'}]))

[14969.857687877915]

Make sure your model can handle this possibility before submitting your model's predict method to the grader.

In [57]:
grader.score.ml__state_model(state_model.predict)

Your score:  0.9999999999999999


## Question 2: simple_features_model

Nursing homes vary greatly in their business characteristics. Some are owned by the government or non-profits while others are run for profit. Some house a few dozen residents while others house hundreds. Some are located within hospitals and may work with more vulnerable populations. We will try to predict which facilities are fined based on their business characteristics.

We'll begin with columns in our DataFrame containing numeric and boolean features. Some of the rows contain null values; estimators cannot handle null values so these must be imputed or dropped. We will create a `Pipeline` containing transformers that process these features, followed by an estimator.

**Note:** When the grader checks your answer, it passes a list of dictionaries to the `predict` or `predict_proba` method of your estimator, not a DataFrame. This means that your model must work with both data types. For this reason, we've provided a custom `ColumnSelectTransformer` for you to use instead `scikit-learn`'s own `ColumnTransformer`.

In [14]:
data.head()

,PROVNUM,PROVNAME,ADDRESS,CITY,STATE,ZIP,PHONE,COUNTY_SSA,COUNTY_NAME,BEDCERT,...,CERTIFICATION,CYCLE_1_DEFS,CYCLE_1_NFROMDEFS,CYCLE_1_NFROMCOMP,CYCLE_1_DEFS_SCORE,CYCLE_1_NUMREVIS,CYCLE_1_REVISIT_SCORE,CYCLE_1_TOTAL_SCORE,CYCLE_1_SURVEY_DATE,CYCLE_2_SURVEY_DATE
0,015010,COOSA VALLEY NURSING FACILITY,315 WEST HICKORY STREET,SYLACAUGA,AL,35150,2562495604,600,Talladega,85,...,Medicare and Medicaid,7,7,0,36,1,0,36,2017-04-06,2016-05-26
1,015012,HIGHLANDS HEALTH AND REHAB,380 WOODS COVE ROAD,SCOTTSBORO,AL,35768,2562183708,350,Jackson,50,...,Medicare and Medicaid,5,5,0,44,1,0,44,2017-03-16,2016-02-04
2,015014,EASTVIEW REHABILITATION & HEALTHCARE CENTER,7755 FOURTH AVENUE SOUTH,BIRMINGHAM,AL,35206,2058330146,360,Jefferson,92,...,Medicare and Medicaid,6,6,0,40,1,0,40,2016-10-20,2015-12-30
3,015015,PLANTATION MANOR NURSING HOME,6450 OLD TUSCALOOSA HIGHWAY P O BOX 97,MC CALLA,AL,35111,2054776161,360,Jefferson,103,...,Medicare and Medicaid,2,2,0,16,1,0,16,2017-03-09,2016-02-11
4,015016,ATHENS HEALTH AND REHABILITATION LLC,611 WEST MARKET STREET,ATHENS,AL,35611,2562321620,410,Limestone,149,...,Medicare and Medicaid,2,2,0,20,1,0,20,2017-06-01,2016-05-12


In [11]:
from sklearn.impute import SimpleImputer

simple_cols = ['BEDCERT', 'RESTOT', 'INHOSP', 'CCRC_FACIL', 'SFF', 'CHOW_LAST_12MOS', 'SPRINKLER_STATUS', 'EXP_TOTAL', 'ADJ_TOTAL']

class ColumnSelectTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, columns):
        self.columns = columns

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return X[self.columns]
        
simple_features = Pipeline([
    ('cst', ColumnSelectTransformer(simple_cols)),
    ('imputer', SimpleImputer())
])

In [13]:
simple_features.fit_transform(data).shape

(13892, 9)

**Note:** The assertion below assumes the output of `noncategorical_features.fit_transform` is a `ndarray`, not a `DataFrame`.)

In [12]:
assert data['RESTOT'].isnull().sum() > 0
assert not np.isnan(simple_features.fit_transform(data)).any()

Now combine the `simple_features` pipeline with an estimator in a new pipeline. Fit `simple_features_model` to the data and submit `simple_features_model.predict_proba` to the grader. You may wish to use cross-validation to tune the hyperparameters of your model.

In [13]:
from sklearn.linear_model import LogisticRegression

simple_features_model = Pipeline([
    ('simple', simple_features),
    ('predictor', LogisticRegression())
])

In [16]:
simple_features_model.fit(data, fine_counts > 0)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Pipeline(memory=None,
         steps=[('simple',
                 Pipeline(memory=None,
                          steps=[('cst',
                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                   'RESTOT',
                                                                   'INHOSP',
                                                                   'CCRC_FACIL',
                                                                   'SFF',
                                                                   'CHOW_LAST_12MOS',
                                                                   'SPRINKLER_STATUS',
                                                                   'EXP_TOTAL',
                                                                   'ADJ_TOTAL'])),
                                 ('imputer',
                                  SimpleImputer(add_indicator=False, copy=True,
                       

In [19]:
def positive_probability(model):
    def predict_proba(X):
        return model.predict_proba(X)[:, 1]
    return predict_proba

grader.score.ml__simple_features(positive_probability(simple_features_model))

Your score:  1.002750512350338


## Question 3: categorical_features

The `'OWNERSHIP'` and `'CERTIFICATION'` columns contain categorical data. We will have to encode the categorical data into numerical features before we pass them to an estimator. Construct one or more pipelines for this purpose. Transformers such as [LabelEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.LabelEncoder.html#sklearn.preprocessing.LabelEncoder) and [OneHotEncoder](https://scikit-learn.org/0.19/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder) may be useful, but you may also want to define your own transformers.

If you used more than one `Pipeline`, combine them with a `FeatureUnion`. As in Question 2, we will combine this with an estimator, fit it, and submit the `predict_proba` method to the grader.

In [23]:
data['OWNERSHIP'].nunique()

13

In [24]:
data['OWNERSHIP'].value_counts() # mamy 13 możliwości w kolumnie "Ownership"

For profit - Corporation                  8091
Non profit - Corporation                  2497
For profit - Partnership                   890
For profit - Individual                    646
Non profit - Church related                493
Government - County                        429
Non profit - Other                         275
Government - Hospital district             227
Government - State                         142
Government - City/county                    77
Government - City                           71
For profit - Limited Liability company      43
Government - Federal                        11
Name: OWNERSHIP, dtype: int64

In [14]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import FeatureUnion

owner_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['OWNERSHIP'])),
    ('onehot', OneHotEncoder(categories='auto', sparse=False))
])

cert_onehot = Pipeline([
    ('cst', ColumnSelectTransformer(['CERTIFICATION'])),
    ('onehot', OneHotEncoder(categories='auto', sparse=False))
])

categorical_features = FeatureUnion([
    ('owner', owner_onehot),
    ('cert', cert_onehot)
])

In [21]:
data_trans = categorical_features.fit_transform(data)
data_trans.shape

(13892, 16)

In [27]:
data_trans.shape #jedna kolumna 'ownership' została przekształcona w 13 kolumn - bo tyle było wariantów w pierwotnej kolumnie

(13892, 16)

In [15]:
assert categorical_features.fit_transform(data).shape[0] == data.shape[0]
assert categorical_features.fit_transform(data).dtype == np.float64
assert not np.isnan(categorical_features.fit_transform(data)).any()

As in the previous question, create a model using the `categorical_features`, fit it to the data, and submit its `predict_proba` method to the grader.

In [16]:
from sklearn.ensemble import RandomForestClassifier

categorical_features_model = Pipeline([
    ('categorical', categorical_features),
    #('lr', LogisticRegression(solver='lbfgs'))
    ('rfc', RandomForestClassifier())
    # add your estimator here
])

In [24]:
categorical_features_model.fit(data, fine_counts > 0)

/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Pipeline(memory=None,
         steps=[('categorical',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('owner',
                                                 Pipeline(memory=None,
                                                          steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['OWNERSHIP'])),
                                                                 ('onehot',
                                                                  OneHotEncoder(categorical_features=None,
                                                                                categories='auto',
                                                                                drop=None,
                                                                                dtype=<class 'numpy.float64'>,
                                                                                handle_unknown='error'

In [48]:
grader.score.ml__categorical_features(positive_probability(categorical_features_model))

Your score:  0.9762628067667403


## Question 4: business_model

Finally, we'll combine `simple_features` and `categorical_features` in a `FeatureUnion`, followed by an estimator in a `Pipeline`. You may want to optimize the hyperparameters of your estimator using cross-validation or try engineering new features (e.g. see [PolynomialFeatures](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html)). When you've assembled and trained your model, pass the `predict_proba` method to the grader.

In [17]:
business_features = FeatureUnion([
    ('simple', simple_features),
    ('categorical', categorical_features)
])

In [62]:
#from sklearn.model_selection import GridSearchCV
#gs = GridSearchCV(LogisticRegression(solver='lbfgs', max_iter=1000),
#                  param_grid={'C': np.arange(1,2.5,0.5)},
#                  cv=4,
#                  n_jobs=4,
#                  verbose=1)

In [18]:
from sklearn.preprocessing import PolynomialFeatures

business_model = Pipeline([
    ('features', business_features),
    ('poly', PolynomialFeatures(2)),
    ('lr', LogisticRegression(solver='lbfgs', max_iter=1000))
    #('gs', gs)
    
])

In [27]:
business_model.fit(data, fine_counts > 0)

/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('simple',
                                                 Pipeline(memory=None,
                                                          steps=[('cst',
                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                   'RESTOT',
                                                                                                   'INHOSP',
                                                                                                   'CCRC_FACIL',
                                                                                                   'SFF',
                                                                                                   'CHOW_LAST_12MOS',
                                      

In [71]:
grader.score.ml__business_model(positive_probability(business_model))

Your score:  1.0432407439120448


## Question 5: survey_results

Surveys reveal safety and health deficiencies at nursing homes that may indicate risk for incidents (and penalties). CMS routinely makes surveys of nursing homes. Build a model that combines the `business_features` of each facility with its cycle 1 survey results, as well as the time between the cycle 1 and cycle 2 survey to predict the cycle 2 total score.

First, let's create a transformer to calculate the difference in time between the cycle 1 and cycle 2 surveys.

In [33]:
data['CYCLE_1_SURVEY_DATE'].head()

0    2017-04-06
1    2017-03-16
2    2016-10-20
3    2017-03-09
4    2017-06-01
Name: CYCLE_1_SURVEY_DATE, dtype: object

In [35]:
data['CYCLE_2_SURVEY_DATE'].head()

0    2016-05-26
1    2016-02-04
2    2015-12-30
3    2016-02-11
4    2016-05-12
Name: CYCLE_2_SURVEY_DATE, dtype: object

In [34]:
pd.to_datetime(data['CYCLE_1_SURVEY_DATE']).head()  #converting strings to datetime objects

0   2017-04-06
1   2017-03-16
2   2016-10-20
3   2017-03-09
4   2017-06-01
Name: CYCLE_1_SURVEY_DATE, dtype: datetime64[ns]

In [36]:
pd.to_datetime(data['CYCLE_1_SURVEY_DATE']) - pd.to_datetime(data['CYCLE_2_SURVEY_DATE'])

0       315 days
1       406 days
2       295 days
3       392 days
4       385 days
          ...   
13887   309 days
13888   336 days
13889   288 days
13890   331 days
13891    98 days
Length: 13892, dtype: timedelta64[ns]

In [49]:
(pd.to_datetime(data['CYCLE_1_SURVEY_DATE']) - pd.to_datetime(data['CYCLE_2_SURVEY_DATE'])).values.reshape(-1,1).astype(float) 

array([[2.72160e+16],
       [3.50784e+16],
       [2.54880e+16],
       ...,
       [2.48832e+16],
       [2.85984e+16],
       [8.46720e+15]])

In [46]:
time_differences = time_differences.reshape(-1,1)

In [47]:
time_differences.shape

(13892, 1)

In [19]:
class TimedeltaTransformer(BaseEstimator, TransformerMixin):
    def __init__(self, t1_col, t2_col):
        self.t1_col = t1_col
        self.t2_col = t2_col

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X)
        return (pd.to_datetime(X[self.t1_col]) - pd.to_datetime(X[self.t2_col])).values.reshape(-1,1).astype(float)

In [20]:
cycle_1_date = 'CYCLE_1_SURVEY_DATE'
cycle_2_date = 'CYCLE_2_SURVEY_DATE'
time_feature = TimedeltaTransformer(cycle_1_date, cycle_2_date)

In the cell below we'll collect the cycle 1 survey features.

In [21]:
cycle_1_cols = ['CYCLE_1_DEFS', 'CYCLE_1_NFROMDEFS', 'CYCLE_1_NFROMCOMP',
                'CYCLE_1_DEFS_SCORE', 'CYCLE_1_NUMREVIS',
                'CYCLE_1_REVISIT_SCORE', 'CYCLE_1_TOTAL_SCORE']
cycle_1_features = ColumnSelectTransformer(cycle_1_cols)

In [55]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression


survey_model = Pipeline([
    ('features', FeatureUnion([
        ('business', business_features),
        ('survey', cycle_1_features),
        ('time', time_feature)
    ])),
    ('poly', PolynomialFeatures(2)),
    ('rfr', RandomForestRegressor(n_estimators=200, min_samples_leaf=100))
    #('lr', LinearRegression()) #regresja liniowa też działała, ale dawała ocenę ok. 0.94, bez ustawiania  paramentrów
])

In [56]:
survey_model.fit(data, cycle_2_score.astype(int))

Pipeline(memory=None,
         steps=[('features',
                 FeatureUnion(n_jobs=None,
                              transformer_list=[('business',
                                                 FeatureUnion(n_jobs=None,
                                                              transformer_list=[('simple',
                                                                                 Pipeline(memory=None,
                                                                                          steps=[('cst',
                                                                                                  ColumnSelectTransformer(columns=['BEDCERT',
                                                                                                                                   'RESTOT',
                                                                                                                                   'INHOSP',
                                               

In [57]:
grader.score.ml__survey_model(survey_model.predict)

Your score:  1.1862365498638858


*Copyright &copy; 2019 The Data Incubator.  All rights reserved.*